# READ ME


# Introduction

The purpose of this assignment to classify images based on 4000 classes. The second part of the assignment involves scoring how simliar two faces are. 

# Prequisites

## Libraries
- python3 with pytorch
- python3 with torchvision

## Computer
- a computer with a GPU, or AWS EC2 instance (trained on g4dn.xlarge for a day)

## Dataset

For manual download:
- Go to [classification](https://www.kaggle.com/c/idl-fall2021-hw2p2s1/data). Select Download All
- Go to [verification](https://www.kaggle.com/c/idl-fall2021-hw2p2s2/data). Select Download All  

For Kaggle API
- Make sure to create a token in your kaggle account. Create a directory `.kaggle` and place the api token in a file `.kaggle/kaggle.json`


**IMPORTANT** Some way or another, make sure the datasets are on the same level as this notebook. You should have the following file names on the **same level**:
```
- train_data/
- val_data/
- test_data/
- verification_pairs_test.txt
- verification_sample/
```
**NOTE**: if you run all the cells under ***Download Data***, you should be fine

# How to Train
This notebook contains all the code necessary to train the model. To train, run all of the cells up the section ***Testing***
starting from the first cell. Once the training is complete, you will want to check the output for the model with the best accuracy. Each model will be of the from model{epoch #}.txt, for example model21.txt had the best validation accuracy so that is what I chose.

# How to Test

## Classification
Run all the cells under ***Classification***

## Verification
Run all the cells under ***Verification***

## Save and submit
Run all the cells under ***Save Submission***

# Model Description

See notebook for more details:
I recreated the resnet34 model with a very similar architecture. The kernel sizes were smaller due to the small 64x64 images. Training involves batch sizes of 128, with a starting learning rate of .1 using SGD. This learning rate exponential decreases by .9 from 0 -> 9 epochs, and then decreases by .8 from 10 -> 15. After 15, it decreases by .7.








# Download Data

In [ ]:
!mkdir .kaggle
import json
token = {"username":"bustin1","key":"914b2f1eda974e0940999f8187a85285"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
 json.dump(token, file)


In [ ]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content

- path is now set to: /content


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
!kaggle competitions download -c idl-fall21-hw2p2s1-face-classification

!kaggle competitions download -c idl-fall21-hw2p2s2-face-verification


 90% 141M/157M [00:00<00:00, 265MB/s]
100% 157M/157M [00:00<00:00, 268MB/s]


In [ ]:
!unzip competitions/idl-fall21-hw2p2s1-face-classification/idl-fall21-hw2p2s1-face-classification.zip

!unzip competitions/idl-fall21-hw2p2s2-face-verification/idl-fall21-hw2p2s2-face-verification.zip


# Imports



In [ ]:
import os
import numpy as np
from PIL import Image

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# TorchVision Dataset and Dataloader

In [ ]:
path_to_train_img_root = 'train_data/'
path_to_val_img_root = 'val_data/'


In [ ]:
bs = 128

In [ ]:
transforms = torchvision.transforms.Compose([
  torchvision.transforms.ToTensor(), 
  torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
  torchvision.transforms.RandomHorizontalFlip(),
])
train_dataset = torchvision.datasets.ImageFolder(root=path_to_train_img_root, 
                                                 transform=torchvision.transforms.ToTensor(),
                                                 )
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, 
                                               shuffle=True, num_workers=2) # change to 8 later on
                                               

In [ ]:
transforms = torchvision.transforms.Compose([
  torchvision.transforms.ToTensor(), 
  torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_dataset = torchvision.datasets.ImageFolder(root=path_to_val_img_root, 
                                               transform=torchvision.transforms.ToTensor(),
                                               )
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, 
                                             shuffle=False, num_workers=2)  # change to 8 later on

TODO: Data augmentation # https://pytorch.org/docs/master/data.html (search for concat and to data augment)

In [ ]:
print("Train dataset stats: ", train_dataset.__len__(), len(train_dataset.classes))
print("Val dataset stats: ", val_dataset.__len__(), len(val_dataset.classes))

Train dataset stats:  380638 4000
Val dataset stats:  8000 4000


# Residual Block (For Resnet 34)

## Block

In [ ]:
class Block(nn.Module):
    def __init__(self, channels, kernel_sizes=[3,3], strides=[2,2], paddings=[1,1], down_sample=None):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels[0], channels[1], kernel_size=kernel_sizes[0], padding=paddings[0], stride=strides[0], bias=False),
            # nn.BatchNorm2d(channels[1], eps=1e-05, momentum=0.1, affine=True),
            nn.PReLU(channels[1]),
            nn.Conv2d(channels[1], channels[2], kernel_size=kernel_sizes[1], padding=paddings[1], stride=strides[1], bias=False),
            # nn.BatchNorm2d(channels[2], eps=1e-05, momentum=0.1, affine=True),
            nn.PReLU(channels[2])
        )
        self.down_sample = down_sample

    def forward(self, x):

        out = self.block(x)
        if self.down_sample:
          x = self.down_sample(x)
        return out + x


# Classification Network


In [ ]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        
        ds1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
            # nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        )

        ds2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=1, stride=1, bias=False),
            # nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        )

        ds3 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=1, stride=1, bias=False),
            # nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
        )


        self.layers = nn.Sequential(
            nn.Conv2d(in_features, 64, kernel_size=3, stride=1, padding=1, bias=False), 
            nn.BatchNorm2d(64, momentum=0.1, affine=True), 
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1),

            # Layer 1
            Block([64, 64, 64], kernel_sizes=[3,3], strides=[1, 1], paddings=[1,1]),
            Block([64, 64, 64], kernel_sizes=[3,3], strides=[1, 1], paddings=[1,1]),
            Block([64, 64, 64], kernel_sizes=[3,3], strides=[1, 1], paddings=[1,1]),

            # Layer 2
            Block([64, 128, 128], kernel_sizes=[3,3], strides=[2, 1], paddings=[1,1], down_sample=ds1), 
            Block([128, 128, 128], kernel_sizes=[3,3], strides=[1, 1], paddings=[1,1]),
            Block([128, 128, 128], kernel_sizes=[3,3], strides=[1, 1], paddings=[1,1]),
            Block([128, 128, 128], kernel_sizes=[3,3], strides=[1, 1], paddings=[1,1]),   

            # Layer 3
            Block([128, 256, 256], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1], down_sample=ds2),
            Block([256, 256, 256], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),
            Block([256, 256, 256], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),
            Block([256, 256, 256], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),
            Block([256, 256, 256], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),
            Block([256, 256, 256], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),

            # Layer 4
            Block([256, 512, 512], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1], down_sample=ds3),
            Block([512, 512, 512], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),
            Block([512, 512, 512], kernel_sizes=[3,3], strides=[1,1], paddings=[1,1]),
            

            nn.AdaptiveAvgPool2d(output_size=(1,1)),
            nn.Flatten()
            
            
        )

        feature_dim = 512
        self.linear_output = nn.Linear(feature_dim, num_classes)    

    def forward(self, x, return_embedding=False):
        embedding = self.layers(x) 
        output = self.linear_output(embedding)
        if return_embedding:
            return embedding, output
        else:
            return output

## Parameters


In [ ]:
numEpochs = 25
in_features = 3 # RGB channels

learningRate = 0.1
weightDecay = 5e-4

num_classes = 4000 # len(train_dataset)


## Build Network Model


In [ ]:
num_classes = len(train_dataset.classes)
network = ClassificationNetwork(in_features, num_classes)
print(network)
network = network.to(device)




In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
scheduler3 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.7)


# Train

In [ ]:
print(scheduler1.get_last_lr())
for epoch in range(numEpochs):

    # Train
    torch.cuda.empty_cache()
    network.train()
    avg_loss = 0.0
    running_loss = 0.0
    import time
    start = time.time()
    num_correct = 0
    for batch_num, (x, y) in enumerate(train_dataloader):

        # push to GPU (hopefully)
        
        x, y = x.to(device), y.to(device)

        # Forward Pass
        
        outputs = network(x)
        loss = criterion(outputs, y.long())

        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item()
        del loss
        num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

        if batch_num % 10 == 9:
            end = time.time()
            running_loss += avg_loss
            print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f} \t Acc: {} \t dt: {}'.format(epoch, batch_num+1, avg_loss/10, num_correct/len(y)/10, end - start))
            start = end
            avg_loss = 0.0
            num_correct = 0
        

    # Validate
    network.eval()
    num_correct = 0
    torch.cuda.empty_cache()
    with torch.no_grad():
        for batch_num, (x, y) in enumerate(val_dataloader):
            x, y = x.to(device), y.to(device)
            outputs = network(x)
            num_correct += (torch.argmax(outputs, axis=1) == y).sum().item()

    total_avg_loss = running_loss / (train_dataset.__len__() / bs)
    if epoch < 9: 
      scheduler1.step()
    elif epoch < 14:
      scheduler2.step()
      torch.save(network.state_dict(), f'model{epoch}.txt')
    else:
      scheduler3.step()
      torch.save(network.state_dict(), f'model{epoch}.txt')

    print('Epoch: {}, Validation Accuracy: {}, Total Avg Loss: {}, lr: {}\n'.format(epoch, num_correct / len(val_dataset), total_avg_loss, scheduler1.get_last_lr()))

print('Done Training')


# Testing

In [ ]:
network = ClassificationNetwork(in_features, num_classes)
network.load_state_dict(torch.load("model21.txt"))
network = network.to(device)

## Classification

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index]).convert('RGB')
        img = torchvision.transforms.ToTensor()(img).to(device)
        img = torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(img)
        img = img.to(device)
        return os.path.basename(self.file_list[index]), img

In [ ]:
def parse_data(datadir):
    img_list = []
    for root, _, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            filei = os.path.join(root, filename)
            img_list.append(filei)

    # construct a dictionary, where key and value correspond to ID and target
    print(img_list)
    print('{}\t{}\n'.format('#Images', len(img_list)))
    return img_list

In [ ]:
path_to_test_img_root = 'test_data/'

In [ ]:
file_list = parse_data(path_to_test_img_root)
test_dataset = ImageDataset(file_list)
test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                             shuffle=False) # change to 8 later on


['test_data/2894.jpg', 'test_data/1997.jpg', 'test_data/1255.jpg', 'test_data/3103.jpg', 'test_data/2840.jpg', 'test_data/3188.jpg', 'test_data/1256.jpg', 'test_data/5233.jpg', 'test_data/7090.jpg', 'test_data/6594.jpg', 'test_data/7222.jpg', 'test_data/2559.jpg', 'test_data/7179.jpg', 'test_data/1340.jpg', 'test_data/3930.jpg', 'test_data/1735.jpg', 'test_data/3652.jpg', 'test_data/5510.jpg', 'test_data/7145.jpg', 'test_data/121.jpg', 'test_data/2690.jpg', 'test_data/2846.jpg', 'test_data/6390.jpg', 'test_data/1924.jpg', 'test_data/1062.jpg', 'test_data/4421.jpg', 'test_data/4769.jpg', 'test_data/6068.jpg', 'test_data/3890.jpg', 'test_data/293.jpg', 'test_data/872.jpg', 'test_data/4394.jpg', 'test_data/7472.jpg', 'test_data/1275.jpg', 'test_data/3844.jpg', 'test_data/1058.jpg', 'test_data/6698.jpg', 'test_data/3227.jpg', 'test_data/4578.jpg', 'test_data/6659.jpg', 'test_data/3121.jpg', 'test_data/5406.jpg', 'test_data/6828.jpg', 'test_data/4751.jpg', 'test_data/7642.jpg', 'test_data/6

In [ ]:
print('Begin Testing')
network.eval()
test_predictions = []
idx_to_class = {v : k for k,v in train_dataset.class_to_idx.items()}
print(idx_to_class)
i = 0
with torch.no_grad():
  for i, (name, x) in enumerate(test_dataloader):
    out = network(x)
    name = "".join([x for x in name])
    label = torch.argmax(out, axis=1).item()
    label = idx_to_class[label]
    test_predictions.append((name, int(label)))

print('Done Testing')

Begin Testing
{0: '0', 1: '1', 2: '10', 3: '100', 4: '1000', 5: '1001', 6: '1002', 7: '1003', 8: '1004', 9: '1005', 10: '1006', 11: '1007', 12: '1008', 13: '1009', 14: '101', 15: '1010', 16: '1011', 17: '1012', 18: '1013', 19: '1014', 20: '1015', 21: '1016', 22: '1017', 23: '1018', 24: '1019', 25: '102', 26: '1020', 27: '1021', 28: '1022', 29: '1023', 30: '1024', 31: '1025', 32: '1026', 33: '1027', 34: '1028', 35: '1029', 36: '103', 37: '1030', 38: '1031', 39: '1032', 40: '1033', 41: '1034', 42: '1035', 43: '1036', 44: '1037', 45: '1038', 46: '1039', 47: '104', 48: '1040', 49: '1041', 50: '1042', 51: '1043', 52: '1044', 53: '1045', 54: '1046', 55: '1047', 56: '1048', 57: '1049', 58: '105', 59: '1050', 60: '1051', 61: '1052', 62: '1053', 63: '1054', 64: '1055', 65: '1056', 66: '1057', 67: '1058', 68: '1059', 69: '106', 70: '1060', 71: '1061', 72: '1062', 73: '1063', 74: '1064', 75: '1065', 76: '1066', 77: '1067', 78: '1068', 79: '1069', 80: '107', 81: '1070', 82: '1071', 83: '1072', 84:

## Verification

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, file_list):
        self.file_list_pairs = []
        self.root = 'verification_data/'
        with open("verification_pairs_test.txt") as f:
          for line in f.readlines():
            img1, img2 = line.split()
            img1_name = self.root+img1
            img2_name = self.root+img2

            img1 = Image.open(img1_name).convert('RGB')
            img1 = torchvision.transforms.ToTensor()(img1).to(device)
            img1 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img1)
            img1_flip = torchvision.transforms.RandomHorizontalFlip(1)(img1)

            img2 = Image.open(img2_name)
            img2 = torchvision.transforms.ToTensor()(img2).to(device)
            img2 = torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img2)
            img2_flip = torchvision.transforms.RandomHorizontalFlip(1)(img2)
            
            self.file_list_pairs.append((img1_name, img1.to(device), img1_flip.to(device), img2_name, img2.to(device), img2_flip.to(device)))

    def __len__(self):
        return len(self.file_list_pairs)

    def __getitem__(self, index):
        img1_name, img1, img1_flip, img2_name, img2, img2_flip = self.file_list_pairs[index]
        
        return self.root + os.path.basename(img1_name), img1, img1_flip, self.root + os.path.basename(img2_name), img2, img2_flip

In [ ]:
def parse_data(datadir):
    img_list = []
    for root, _, filenames in os.walk(datadir):  #root: median/1
        for filename in filenames:
            filei = os.path.join(root, filename)
            img_list.append(filei)

    # construct a dictionary, where key and value correspond to ID and target
    print('{}\t{}\n'.format('#Images', len(img_list)))
    return img_list

In [ ]:
path_to_test_img_root = 'verification_data/'

In [ ]:
file_list = parse_data(path_to_test_img_root)
test_dataset = ImageDataset(file_list)
test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                             shuffle=False) # change to 8 later on

#Images	69097



In [ ]:
print(file_list[0])
counter=0
with open('verification_pairs_test.txt', 'r') as f:
  for line in f:
    counter += 1
print('Number of rows in submission: ' + counter)

verification_data/verification_data/00061520.jpg
51835


## flip + output + euclean

In [ ]:
print('Begin Testing')
network.eval()
test_predictions2 = []
print(len(test_dataloader))
with torch.no_grad():
  for i, (img1_name, img1, img1_flip, img2_name, img2, img2_flip) in enumerate(test_dataloader):
    _, out1 = network(img1, return_embedding=True)
    _, out2 = network(img2, return_embedding=True)

    img1_name = "".join([x for x in img1_name])
    img2_name = "".join([x for x in img2_name])

    _, out3 = network(img1_flip, return_embedding=True)
    _, out4 = network(img2_flip, return_embedding=True)

    inter1 = nn.Softmax()(torch.cat((out1, out3), dim=1))
    inter2 = nn.Softmax()(torch.cat((out2, out4), dim=1))
    label = torch.abs(torch.nn.CosineSimilarity()(out1, out2))

    test_predictions2.append((img1_name, img2_name, label.item()))
    if i % 100 == 0:
      print(i)

print('Done Testing')

Begin Testing
51835
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

# Save Submission

In [ ]:
test_predictions = [(a,b,abs(c)) for a,b,c in test_predictions]

In [ ]:
print(len(test_predictions))
print(test_predictions[0])

51835
('verification_data/00020839.jpg', 'verification_data/00035322.jpg', 0.42573776841163635)


In [ ]:
print(len(test_predictions2))
print(test_predictions2[0])

51835
('verification_data/00020839.jpg', 'verification_data/00035322.jpg', 0.5630297064781189)


In [ ]:
print(len(test_predictions3))
print(test_predictions3[0])

51835
('verification_data/00020839.jpg', 'verification_data/00035322.jpg', 0.5476054549217224)


In [ ]:
import csv
with open('submission.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['id','category'])
    for row in test_predictions3:
        id = row[0] + ' ' + row[1]
        category = row[2]
        csv_out.writerow((id, category))

NameError: ignored

In [ ]:
import pandas as pd
df = pd.read_csv('submission.csv')
print(df)

                                                      id  category
0      verification_data/00020839.jpg verification_da...  0.563030
1      verification_data/00002921.jpg verification_da...  0.364008
2      verification_data/00011732.jpg verification_da...  0.218998
3      verification_data/00052778.jpg verification_da...  0.482578
4      verification_data/00053948.jpg verification_da...  0.427123
...                                                  ...       ...
51830  verification_data/00041961.jpg verification_da...  0.354976
51831  verification_data/00060107.jpg verification_da...  0.182393
51832  verification_data/00003205.jpg verification_da...  0.245838
51833  verification_data/00068054.jpg verification_da...  0.506417
51834  verification_data/00046783.jpg verification_da...  0.275933

[51835 rows x 2 columns]


In [ ]:
import pandas as pd
df = pd.read_csv('submission.csv')
print(df)

                                                      id  category
0      verification_data/00020839.jpg verification_da...  0.425738
1      verification_data/00002921.jpg verification_da...  0.146640
2      verification_data/00011732.jpg verification_da...  0.067483
3      verification_data/00052778.jpg verification_da...  0.334864
4      verification_data/00053948.jpg verification_da...  0.246022
...                                                  ...       ...
51830  verification_data/00041961.jpg verification_da...  0.156179
51831  verification_data/00060107.jpg verification_da...  0.020182
51832  verification_data/00003205.jpg verification_da...  0.090006
51833  verification_data/00068054.jpg verification_da...  0.344626
51834  verification_data/00046783.jpg verification_da...  0.036033

[51835 rows x 2 columns]


In [ ]:
input1 = F.relu_(torch.randn(100))
input2 = F.relu_(torch.randn(100))
output = F.cosine_similarity(input1, input2, dim=0)
print(output.item())


0.351718008518219


In [ ]:
!kaggle competitions submit -c idl-fall21-hw2p2s2-face-verification -f submission.csv -m "horizontal flip + softmax from output"


100% 4.09M/4.09M [00:03<00:00, 1.18MB/s]
Successfully submitted to IDL-Fall21-HW2P2S2-Face Verification